In [1]:
%matplotlib inline
import datetime
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import re
import time
from elasticsearch import Elasticsearch, helpers
import csv

In [2]:


bfile=open("boinc_job.csv","w")
pfile=open("panda_job.csv","w")
outb = csv.writer(bfile, delimiter=',',quoting=csv.QUOTE_ALL)
outp = csv.writer(pfile, delimiter=",",quoting=csv.QUOTE_ALL)

last_days=50
es = Elasticsearch([{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=60)

def time_filter(indices,pat=r'jobs_archive_', last_days=1):
    if last_days == 0:
        return ["jobs_archive_*"]
    today = datetime.date.today()
    filtered = []
    datefmt = '%Y-%m-%d'
    for i in indices:
        day = re.sub(pat, '', i).rstrip()
        day = datetime.datetime.strptime(day, datefmt).date()
        diff = today - day
        if diff.days < last_days:
            filtered.append(i.rstrip())
    return filtered

def get_indices(index="jobs_archive_*",pat=r'jobs_archive_',last_days=last_days):
    #indices_ja = es.cat.indices(index="jobs_archive_*", h="index", request_timeout=600).split('\n')
    indices = es.cat.indices(index=index, h="index", request_timeout=600).split('\n')
    indices = sorted(indices)
    indices = [x for x in indices if x != '' and not x.endswith('_reindexed')]
    #print("indices is",indices)
    if index=="boinc":
        ind=indices
    else:
        ind = time_filter(indices,pat=pat,last_days=last_days)
        print ("number of indices:", len(ind))
    ind = ','.join(ind)
    print(ind)
    return ind
ind_ja=get_indices()
ind_boinc=get_indices(index="boinc",pat=r'boinc_')

number of indices: 50
jobs_archive_2017-12-10,jobs_archive_2017-12-11,jobs_archive_2017-12-12,jobs_archive_2017-12-13,jobs_archive_2017-12-14,jobs_archive_2017-12-15,jobs_archive_2017-12-16,jobs_archive_2017-12-17,jobs_archive_2017-12-18,jobs_archive_2017-12-19,jobs_archive_2017-12-20,jobs_archive_2017-12-21,jobs_archive_2017-12-22,jobs_archive_2017-12-23,jobs_archive_2017-12-24,jobs_archive_2017-12-25,jobs_archive_2017-12-26,jobs_archive_2017-12-27,jobs_archive_2017-12-28,jobs_archive_2017-12-29,jobs_archive_2017-12-30,jobs_archive_2017-12-31,jobs_archive_2018-01-01,jobs_archive_2018-01-02,jobs_archive_2018-01-03,jobs_archive_2018-01-04,jobs_archive_2018-01-05,jobs_archive_2018-01-06,jobs_archive_2018-01-07,jobs_archive_2018-01-08,jobs_archive_2018-01-09,jobs_archive_2018-01-10,jobs_archive_2018-01-11,jobs_archive_2018-01-12,jobs_archive_2018-01-13,jobs_archive_2018-01-14,jobs_archive_2018-01-15,jobs_archive_2018-01-16,jobs_archive_2018-01-17,jobs_archive_2018-01-18,jobs_archive_2018-

In [3]:
my_query_ja={
    "size": 0,
    "_source": ["modificationhost", "cpuconsumptiontime","nevents","jeditaskid","actualcorecount","wall_time","nevents","pilotid"],
  "query": {
    "bool": {
      "must": [
        {
          "term": {"jobstatus": "finished"},
          "term": {"computingsite":"BOINC_MCORE"}
        }
      ]
    }
  }
}
my_query_boinc={
    "size": 0,
    "_source": ["result_app_version_id","batch","cpu_time","host_name","elapsed_time","name","host_flops","host_ncpus"],
  "query": {
    "bool": {
      "must": [
        {
          "term": {"outcome":1},
          "term": {"result_app_id":14}
        }
      ]
    }
  }
}


def list_res(res):
    count=0
    for r in res:
        if count<=10:
            print(r)
            count+=1





In [4]:
#get_avg_boinc_power_by_task(tasks,res)
def get_ja_info(res):
    d=["host","name","ncore","panda_cputime","panda_walltime","panda_cpu_eff","nevents","task"]
    outp.writerow(d)
    for r in res:
        if "actualcorecount" in r["_source"] and r["_source"]["actualcorecount"]:
            taskid=r["_source"]["jeditaskid"]
            cputime=r["_source"]["cpuconsumptiontime"]
            walltime=r["_source"]["wall_time"]*r["_source"]["actualcorecount"]
            nevents=r["_source"]["nevents"]
            ncore=r["_source"]["actualcorecount"]
            pilotid=r["_source"]["pilotid"]
            name=pilotid
            host=r["_source"]["modificationhost"]
            
            if type(host)!=str or host=="":
                continue
            host=re.sub(".*@","",host)
            if cputime>2000*nevents or cputime<=0:
                continue
            if walltime>4000*nevents*ncore or walltime<=0:
                continue
            cpu_eff=np.round(cputime/walltime,2)
            if cpu_eff<1:
                d=[host,name,ncore,cputime,walltime,cpu_eff,nevents,taskid]
                outp.writerow(d)
def get_boinc_info(res):
    d=["av","host","name","boinc_cputime","boinc_walltime","host_flops","host_ncpu","task"]
    outb.writerow(d)
    for r in res:
        taskid=r["_source"]["batch"]
        name=r["_source"]["name"]
        host=r["_source"]["host_name"]
        cputime=r["_source"]["cpu_time"]
        walltime=r["_source"]["elapsed_time"]
        av=r["_source"]["result_app_version_id"]
        host_flops=r["_source"]["host_flops"]
        host_ncpu=r["_source"]["host_ncpus"]
        if walltime<=0 or cputime<=0:
            continue    
        if host=="":
            continue
        d=(av,host,name,cputime,walltime,host_flops,host_ncpu,taskid)
        outb.writerow(d)


res_ja = helpers.scan(es, query=my_query_ja, index=ind_ja, scroll='5m', timeout="5m", size=1000)
get_ja_info(res_ja)

res_boinc = helpers.scan(es, query=my_query_boinc, index=ind_boinc, scroll='5m', timeout="5m", size=1000) 
get_boinc_info(res_boinc)    
